# AST Transform Routines

In [1]:
%load_ext literary.notebook

import pytest

from .utils import unparse_ast

In [2]:
import ast
import logging
import typing as tp

from traitlets import Bool, Unicode
from traitlets.config import Configurable

In [3]:
logger = logging.getLogger(__name__)

In [4]:
T = tp.TypeVar("T")

## IPython Magic Transformer

Many notebooks will contain IPython magics. It is important that generated modules do not contain these magics unless they are explicitly permitted with the `literary.IPythonTransformer.error_if_python` option is disabled.

In [5]:
class IPythonTransformer(Configurable):
    """Node transformer which operates upon ast.Module nodes to check for IPython
    statements.
    """

    error_if_ipython = Bool(default_value=True).tag(config=True)

    def visit(self, node: ast.Module) -> ast.Module:
        """Transform a Python module AST node

        :param node: ast.Module object
        :return: ast.Module object
        """
        for descendant in ast.walk(node):
            if not isinstance(descendant, ast.Call):
                continue

            if not isinstance(descendant.func, ast.Name):
                continue

            if descendant.func.id == "get_ipython":
                msg = (
                    "`get_ipython` cannot be transpiled to pure-Python. "
                    "Check for magics in exported code cells"
                )
                if self.error_if_ipython:
                    raise ValueError(msg)
                else:
                    logger.warning(msg)

        return node

Let's create a demo transformer:

In [6]:
ipython_transformer = IPythonTransformer(error_if_ipython=True)

Generate an AST node that should fail upon visitation

In [7]:
node = ast.parse("get_ipython().magic('matplotlib inline')")

Now we run the test and raise an exception if it doesn't fail with a `ValueError`

In [8]:
with pytest.raises(ValueError):
    ipython_transformer.visit(node)

## Class Monkeypatch Transformer
In Jupyter notebooks it is not always convenient to write long definitions; e.g. class definitions can be composed of several nontrivial member functions and attributes. Here we define a transformer to rejoin disparate definitions such that the generated AST tree is performant, and will generate code that lives under a single class statement.

In [9]:
class PatchTransformer(Configurable):
    """Node transformer which operates upon ast.Module nodes to handle patches."""

    patch_decorator_id = Unicode("patch").tag(config=True)

    def _is_patch_decorator(self, node) -> bool:
        """Return True if the given ast node is a patch() decorator

        :param node: ast node
        :return:
        """
        return (
            isinstance(node, ast.Call)
            and isinstance(node.func, ast.Name)
            and node.func.id == self.patch_decorator_id
        )

    def _apply_patches_to_node(self, decorated, classes: tp.Dict[str, tp.Any]):
        """Append patch functions to body of patched class.

        :param decorated: decorated function node
        :param classes: mapping of name to class node
        :return: whether node should be removed from original AST location
        """
        patch_decorators = []
        other_decorators = []

        for node in decorated.decorator_list:
            if self._is_patch_decorator(node):
                if other_decorators:
                    raise ValueError(
                        "Applying decorators to a patched function is not "
                        "safe at runtime, and is hence prohibited."
                    )
                patch_decorators.append(node)
            else:
                other_decorators.append(node)

        if not patch_decorators:
            return False

        logger.info(f"Found patch function {ast.dump(decorated)}")
        decorated.decorator_list = other_decorators

        # Copy decorated node to destination body
        for node in reversed(patch_decorators):
            (patched_name_node,) = node.args
            patched_node = classes[patched_name_node.id]
            patched_node.body.append(decorated)
            logger.debug(f"Patching {patched_node.name} with {ast.dump(decorated)}")

        return True

    def _transform_module_body(self, nodes: tp.Iterable[T]) -> tp.Iterable[T]:
        """Transform the nodes of an ast.Module to handle patches.

        :param nodes: iterable of ast nodes
        :return:
        """
        classes = {}
        for child in nodes:
            if isinstance(child, ast.ClassDef):
                classes[child.name] = child

            # If there were patch decorators, don't keep this node
            if hasattr(child, "decorator_list") and self._apply_patches_to_node(
                child, classes
            ):
                continue

            yield child

    def visit(self, node: ast.Module) -> ast.Module:
        """Transform a Python module AST node.

        :param node: ast.Module object
        :return: ast.Module object
        """
        node.body = [*self._transform_module_body(node.body)]
        ast.fix_missing_locations(node)
        return node

We can demonstrate this on an example class

In [10]:
transformer = PatchTransformer(patch_decorator_id="patch")

In [11]:
node = ast.parse(
    """
class TestClass:
    pass

@patch(TestClass)
def method_a(self):
    pass

@patch(TestClass)
def method_b(self):
    pass
"""
)

In [12]:
new_node = transformer.visit(node)

In [13]:
print(unparse_ast(new_node))

class TestClass:
    pass

    def method_a(self):
        pass

    def method_b(self):
        pass
